# BUSINESS ANALYTICS OF A BIBYCLES COMPANY

# 1. Introduction

## Report goal
There is a small fictional cycling company that sells bicycles in different countries around the world. They would like to increase their revenue and improve their business. To do this, they provided a dataset with all their transactions in the years 2013-2014. This dataset includes product-level information about sales, profits, market segments, and geographic distribution.

The main idea is to use these details to gain new insights to create or improve the marketing and sales strategy.

Specifically, it will be analyze:
- the sales trends over time using a line chart to highlight seasonality or trends;
- the sales difference across regions using a heat map or bar chart;
- the top-selling products using visualization charts (pie chart or bar chart);
- the relationship between discounts and sales or profits using a scatter plot or by creating a linear model regression.

## Database description
The database is composed by 700 rows and 16 columns:
- segment:
- country:
- product:
- discount band:
- unit sold:
- manufactoring price:
- sale price:
- gross sales:
- discount:
- sales:
- COGS:
- profit:
- date: 
- month number
- month name
- year


# 2. Loading and inspection of the dataset

Data import with API

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("atharvaarya25/financials")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\matza\.cache\kagglehub\datasets\atharvaarya25\financials\versions\1


In [212]:
import pandas as pd

df = pd.read_csv(f"{path}/Financials.csv")
df.head()

,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Month Number,Month Name,Year
0,Government,Canada,Carretera,None,"$1,618.50",$3.00,$20.00,"$32,370.00",$-,"$32,370.00","$16,185.00","$16,185.00",01/01/2014,1,January,2014
1,Government,Germany,Carretera,None,"$1,321.00",$3.00,$20.00,"$26,420.00",$-,"$26,420.00","$13,210.00","$13,210.00",01/01/2014,1,January,2014
2,Midmarket,France,Carretera,None,"$2,178.00",$3.00,$15.00,"$32,670.00",$-,"$32,670.00","$21,780.00","$10,890.00",01/06/2014,6,June,2014
3,Midmarket,Germany,Carretera,None,$888.00,$3.00,$15.00,"$13,320.00",$-,"$13,320.00","$8,880.00","$4,440.00",01/06/2014,6,June,2014
4,Midmarket,Mexico,Carretera,None,"$2,470.00",$3.00,$15.00,"$37,050.00",$-,"$37,050.00","$24,700.00","$12,350.00",01/06/2014,6,June,2014


Dataset overview

In [79]:
# copy of the dataset
df_copy = df.copy(deep=True)

In [80]:
# number or rows and columns
df.shape

(700, 16)

In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Segment                700 non-null    object
 1   Country                700 non-null    object
 2    Product               700 non-null    object
 3    Discount Band         700 non-null    object
 4    Units Sold            700 non-null    object
 5    Manufacturing Price   700 non-null    object
 6    Sale Price            700 non-null    object
 7    Gross Sales           700 non-null    object
 8    Discounts             700 non-null    object
 9     Sales                700 non-null    object
 10   COGS                  700 non-null    object
 11   Profit                700 non-null    object
 12  Date                   700 non-null    object
 13  Month Number           700 non-null    int64 
 14   Month Name            700 non-null    object
 15  Year                   

# 3. Cleaning of the dataset

Cleaning the column names

In [82]:
# check the column names
df.columns

Index(['Segment', 'Country', ' Product ', ' Discount Band ', ' Units Sold ',
       ' Manufacturing Price ', ' Sale Price ', ' Gross Sales ', ' Discounts ',
       '  Sales ', ' COGS ', ' Profit ', 'Date', 'Month Number',
       ' Month Name ', 'Year'],
      dtype='object')

In [83]:
# column names with anomalous spaces
columns_whiteSpaceName = df.columns[df.columns.str.startswith(" ") | df.columns.str.endswith(" ")]
columns_whiteSpaceName

Index([' Product ', ' Discount Band ', ' Units Sold ', ' Manufacturing Price ',
       ' Sale Price ', ' Gross Sales ', ' Discounts ', '  Sales ', ' COGS ',
       ' Profit ', ' Month Name '],
      dtype='object')

In [213]:
# remove white spaces
df.columns = df.columns.str.strip()

Cleaning the column values

In [124]:
# columns type
df.dtypes

Segment                object
Country                object
Product                object
Discount Band          object
Units Sold             object
Manufacturing Price    object
Sale Price             object
Gross Sales            object
Discounts              object
Sales                  object
COGS                   object
Profit                 object
Date                   object
Month Number            int64
Month Name             object
Year                    int64
dtype: object

In [125]:
df.head(3)

,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Month Number,Month Name,Year
0,Government,Canada,Carretera,None,"$1,618.50",$3.00,$20.00,"$32,370.00",$-,"$32,370.00","$16,185.00","$16,185.00",01/01/2014,1,January,2014
1,Government,Germany,Carretera,None,"$1,321.00",$3.00,$20.00,"$26,420.00",$-,"$26,420.00","$13,210.00","$13,210.00",01/01/2014,1,January,2014
2,Midmarket,France,Carretera,None,"$2,178.00",$3.00,$15.00,"$32,670.00",$-,"$32,670.00","$21,780.00","$10,890.00",01/06/2014,6,June,2014


In [214]:
# remove $ and space
columns_withDollar = ['Units Sold', 'Manufacturing Price', 'Sale Price', 'Gross Sales', 'Discounts', 'Sales', 'COGS', 'Profit']

for col in columns_withDollar:
    if df[col].dtype == 'object':
        df[col] = df[col].str.replace('$', '')
        df[col] = df[col].str.replace(',', '')
        df[col] = df[col].apply(lambda x: x.strip())

df.head(3)

,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Month Number,Month Name,Year
0,Government,Canada,Carretera,None,1618.50,3.00,20.00,32370.00,-,32370.00,16185.00,16185.00,01/01/2014,1,January,2014
1,Government,Germany,Carretera,None,1321.00,3.00,20.00,26420.00,-,26420.00,13210.00,13210.00,01/01/2014,1,January,2014
2,Midmarket,France,Carretera,None,2178.00,3.00,15.00,32670.00,-,32670.00,21780.00,10890.00,01/06/2014,6,June,2014


In [215]:
# changing Discount column
df['Discounts'] = df['Discounts'].replace('-', 0)

In [181]:
# check again for unusual symbols
for col in columns_withDollar:
    print(f"Check column: {col}")
    print(df[col][df[col].str.contains(r'^\s*$|^-|^\$|\(|\)', na=False)].unique())

Check column: Units Sold
[]
Check column: Manufacturing Price
[]
Check column: Sale Price
[]
Check column: Gross Sales
[]
Check column: Discounts
[]
Check column: Sales
[]
Check column: COGS
[]
Check column: Profit
['-' '(4533.75)' '(3740.00)' '(2981.25)' '(1076.25)' '(880.00)'
 '(1008.75)' '(2217.50)' '(2380.00)' '(6887.50)' '(4968.75)' '(9375.00)'
 '(13173.75)' '(7826.25)' '(6168.75)' '(3727.50)' '(7700.00)' '(5570.00)'
 '(11606.25)' '(17481.25)' '(3543.75)' '(13187.50)' '(5481.25)'
 '(25841.25)' '(11115.00)' '(13530.00)' '(11970.00)' '(14370.00)'
 '(2557.50)' '(18967.50)' '(4342.50)' '(21358.75)' '(4847.50)'
 '(27693.75)' '(12538.75)' '(8286.25)' '(24160.00)' '(21560.00)'
 '(23870.00)' '(17808.75)' '(18663.75)' '(12787.50)' '(35262.50)'
 '(19687.50)' '(35550.00)' '(16142.50)' '(38046.25)' '(14918.75)'
 '(9116.25)' '(33522.50)' '(40617.50)' '(7590.00)']


In [196]:
(df['Profit']=='-').sum()
# there are only 5 values in Profit column with "-", they will be considered as NaN values

np.int64(5)

In [216]:
# change Profit column
df['Profit'] = df['Profit'].str.strip('()')
df['Profit'] = df['Profit'].replace('-', None)


In [217]:
# change column types
for col in columns_withDollar:
    if df[col].dtype == 'object':
        df[col] = df[col].astype(float)

In [218]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Segment              700 non-null    object 
 1   Country              700 non-null    object 
 2   Product              700 non-null    object 
 3   Discount Band        700 non-null    object 
 4   Units Sold           700 non-null    float64
 5   Manufacturing Price  700 non-null    float64
 6   Sale Price           700 non-null    float64
 7   Gross Sales          700 non-null    float64
 8   Discounts            700 non-null    float64
 9   Sales                700 non-null    float64
 10  COGS                 700 non-null    float64
 11  Profit               695 non-null    float64
 12  Date                 700 non-null    object 
 13  Month Number         700 non-null    int64  
 14  Month Name           700 non-null    object 
 15  Year                 700 non-null    int

In [223]:
df = df.dropna()

# change column types
for col in columns_withDollar:
    if df[col].dtype == 'float':
        df[col] = df[col].astype(int)

In [229]:
# controll of the values
round(df[columns_withDollar].describe(), 2)

# It seems not to be strange values, cleaning data completed

,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit
count,695.00,695.00,695.00,695.00,695.00,695.00,695.00,695.00
mean,1609.58,96.75,118.38,182791.32,13193.37,169597.38,145290.19,26544.08
std,867.79,108.61,137.27,255038.27,23037.42,237441.03,204428.18,41515.20
min,200.00,3.00,7.00,1799.00,0.00,1655.00,918.00,285.00
25%,905.00,5.00,12.00,17325.00,798.00,15851.00,7480.00,4013.00
50%,1545.00,10.00,20.00,37080.00,2559.00,34736.00,21980.00,11135.00
75%,2230.50,250.00,300.00,280850.00,16164.50,261575.00,245905.00,23918.50
max,4492.00,260.00,350.00,1207500.00,149677.00,1159200.00,950625.00,262200.00


# 4. Exploratory Data Analysis

# 5. Statistical analysis

# 6. Conclusion